In [1]:
import pandas as pd

df = pd.read_csv('C:\dataset\IMDB Dataset.csv')
df.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [2]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [3]:
df.loc[0, 'review'][-50:]

'ats if you can get in touch with your darker side.'

In [4]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [5]:
preprocessor(df.loc[0, 'review'][-50:])

'ats if you can get in touch with your darker side '

In [6]:
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :) :( :)'

In [7]:
df['review'] = df['review'].apply(preprocessor)

In [8]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):
    return text.split()


def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [9]:
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [10]:
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [11]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\TANMAYI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:]
if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

In [13]:
#transform text data into TF-IDF vector
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf=TfidfVectorizer(strip_accents=None,
                      lowercase=False,
                      preprocessor=None,
                      tokenizer=tokenizer_porter,
                      use_idf=True,
                      norm='l2',
                      smooth_idf=True)

y=df.sentiment.values
X=tfidf.fit_transform(df.review)

In [14]:
#Document Classification using logistic Regression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1,test_size = 0.5, shuffle = False)

In [15]:
import pickle
from sklearn.linear_model import LogisticRegressionCV

clf=LogisticRegressionCV(cv=5, scoring='accuracy',random_state=0,n_jobs=-1,verbose=3,max_iter=300).fit(X_train,y_train)

saved_model=open('saved_model.sav','wb')
pickle.dump(clf, saved_model)
saved_model.close()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  2.1min remaining:  3.2min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.2min finished


In [16]:
print("Review: ", X_train[1:2], "\n")
print("Sentiment: ", y_train[1])

Review:    (0, 19076)	0.05692772866076392
  (0, 64500)	0.06604914707653949
  (0, 62967)	0.10091698951616648
  (0, 21968)	0.05310425418815358
  (0, 17089)	0.11455615469467624
  (0, 44132)	0.1730434168816232
  (0, 23886)	0.08553328303488324
  (0, 28652)	0.3335851221435916
  (0, 47239)	0.15968664020527257
  (0, 14447)	0.08430379183162137
  (0, 48398)	0.14998820263821241
  (0, 35961)	0.0961843556310927
  (0, 47386)	0.12614251319939748
  (0, 50044)	0.04340618038374713
  (0, 18425)	0.09379745672100832
  (0, 64806)	0.0437753121734598
  (0, 60172)	0.08785407120528048
  (0, 53655)	0.0780909726232393
  (0, 19508)	0.07767394381977552
  (0, 66215)	0.09052978737167931
  (0, 64639)	0.039854926392073345
  (0, 52897)	0.06029579529086435
  (0, 22739)	0.08565256157082338
  (0, 14194)	0.04503758587028675
  (0, 53081)	0.03857093430367422
  :	:
  (0, 27261)	0.05507693274379067
  (0, 10600)	0.046772737656584434
  (0, 30588)	0.0658546966736318
  (0, 3187)	0.05810978245680153
  (0, 46835)	0.025685467647607357

In [17]:
#model evaluation
filename = 'saved_model.sav'
saved_clf = pickle.load(open(filename, 'rb'))

In [18]:
saved_clf.score(X_test, y_test)

C:\Users\TANMAYI\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:2260: ChangedBehaviorWarning: The long-standing behavior to use the accuracy score has changed. The scoring parameter is now used. This warning will disappear in version 0.22.
  ChangedBehaviorWarning)


0.89472